In [5]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import re
import time

# Verificar GPU
device = 0 if torch.cuda.is_available() else -1
print(f"Usando dispositivo: {'GPU' if device == 0 else 'CPU'}")
if device == 0:
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Cargar modelo
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Crear pipeline con GPU
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    device=device,
    truncation=True,
    max_length=512
)

# Función para extraer número de estrellas
def extract_stars(label):
    """Extrae el número de estrellas del label (ej: '2 stars' -> 2)"""
    match = re.search(r'(\d+)', label)
    return int(match.group(1)) if match else None

# Cargar CSV
print("Cargando CSV...")
df = pd.read_csv('combined_news_data.csv')
print(f"Total de filas en CSV: {len(df)}")

# Filtrar datos
print("Filtrando datos...")
df_filtered = df[df['text'].str.len() > 100].copy()
print(f"Filas con texto > 100 caracteres: {len(df_filtered)}")

df_filtered = df_filtered[df_filtered['date_parsed'].notna()].copy()
print(f"Filas con fecha válida: {len(df_filtered)}")

# Preparar listas para almacenar resultados
results = []
total_rows = len(df_filtered)
errors = 0
start_time = time.time()

print(f"\nProcesando {total_rows} noticias...")
print("Iniciando procesamiento...")

# Procesar cada fila
for idx, row in df_filtered.iterrows():
    try:
        # Analizar sentimiento
        resultado = sentiment_pipeline(row['text'])[0]
        
        # Extraer información
        stars = extract_stars(resultado['label'])
        score = resultado['score']
        
        # Agregar a resultados
        results.append({
            'id': row['id'],
            'title': row['title'],
            'country': row['country'],
            'date': row['date_parsed'],
            'url': row['url'],
            'stars': stars,
            'score': score
        })
        
    except Exception as e:
        errors += 1
        # Agregar fila con valores nulos en caso de error  
        results.append({
            'id': row['id'],
            'title': row['title'],
            'country': row['country'],
            'date': row['date_parsed'],
            'url': row['url'],
            'stars': None,
            'score': None
        })
    
    # Print cada 1000 elementos con estimación de tiempo
    if len(results) % 1000 == 0:
        elapsed_time = time.time() - start_time
        processed = len(results)
        remaining = total_rows - processed
        
        if processed > 0:
            time_per_item = elapsed_time / processed
            estimated_remaining = time_per_item * remaining
            
            # Convertir a minutos y segundos
            elapsed_mins = int(elapsed_time // 60)
            elapsed_secs = int(elapsed_time % 60)
            remaining_mins = int(estimated_remaining // 60)
            remaining_secs = int(estimated_remaining % 60)
            
            print(f"Procesadas {processed}/{total_rows} noticias | "
                  f"Errores: {errors} | "
                  f"Tiempo transcurrido: {elapsed_mins}m {elapsed_secs}s | "
                  f"Tiempo estimado restante: {remaining_mins}m {remaining_secs}s")

# Crear DataFrame con resultados
df_results = pd.DataFrame(results)

# Guardar CSV
#output_filename = 'sentiment_analysis_results.csv'
#df_results.to_csv(output_filename, index=False)

# Tiempo total
total_time = time.time() - start_time
total_mins = int(total_time // 60)
total_secs = int(total_time % 60)

print(f"\n¡Completado!")
print(f"Tiempo total: {total_mins}m {total_secs}s")
print(f"Archivo guardado como: {output_filename}")
print(f"Total de noticias procesadas: {len(df_results)}")
print(f"Noticias con análisis exitoso: {df_results['stars'].notna().sum()}")
print(f"Errores encontrados: {errors}")

# Mostrar estadísticas
if len(df_results) > 0:
    print(f"\nEstadísticas de sentimientos:")
    print(df_results['stars'].value_counts().sort_index())
    print(f"\nScore promedio: {df_results['score'].mean():.3f}")

Usando dispositivo: GPU
GPU: NVIDIA GeForce RTX 3060


Device set to use cuda:0


Cargando CSV...
Total de filas en CSV: 94282
Filtrando datos...
Filas con texto > 100 caracteres: 93814
Filas con fecha válida: 93813

Procesando 93813 noticias...
Iniciando procesamiento...
Procesadas 1000/93813 noticias | Errores: 0 | Tiempo transcurrido: 0m 19s | Tiempo estimado restante: 30m 33s
Procesadas 2000/93813 noticias | Errores: 0 | Tiempo transcurrido: 0m 39s | Tiempo estimado restante: 30m 24s
Procesadas 3000/93813 noticias | Errores: 0 | Tiempo transcurrido: 0m 59s | Tiempo estimado restante: 29m 51s
Procesadas 4000/93813 noticias | Errores: 0 | Tiempo transcurrido: 1m 19s | Tiempo estimado restante: 29m 52s
Procesadas 5000/93813 noticias | Errores: 0 | Tiempo transcurrido: 1m 39s | Tiempo estimado restante: 29m 34s
Procesadas 6000/93813 noticias | Errores: 0 | Tiempo transcurrido: 1m 58s | Tiempo estimado restante: 28m 54s
Procesadas 7000/93813 noticias | Errores: 0 | Tiempo transcurrido: 2m 16s | Tiempo estimado restante: 28m 8s
Procesadas 8000/93813 noticias | Errores

In [6]:
df_results

,id,title,country,date,url,stars,score
0,1,Pandemia y salud mental: Ministerio de Ciencia...,chile,2021-04-28 20:00:00,https://www.latercera.com/que-pasa/noticia/pan...,4,0.424832
1,2,"Podcast, cápsulas animadas y libros invitan a ...",chile,2020-07-02 20:00:00,https://www.elrepuertero.cl/noticia/tecnologia...,4,0.457038
2,3,Ministerio de Ciencia presenta plataforma con ...,chile,2021-05-02 20:00:00,https://www.aconcaguadigital.cl/ministerio-de-...,4,0.463792
3,4,"Salud mental en niños, tenemos que estar atentos",chile,2021-06-28 20:00:00,https://www.laestrellachiloe.cl/impresa/2021/0...,2,0.328486
4,5,"¿Quién tiene peor salud mental, la Generación ...",chile,2023-09-13 21:00:00,https://www.latercera.com/que-pasa/noticia/qui...,2,0.315747
...,...,...,...,...,...,...,...
93808,94278,Abusos marcan a Rocío Nahle,mexico,2019-05-15 20:00:00,https://www.ejecentral.com.mx/abusos-marcan-a-...,4,0.348778
93809,94279,"Yunior García, el artista que desafía el gobie...",mexico,2021-11-12 21:00:00,https://www.bbc.com/mundo/noticias-america-lat...,1,0.503947
93810,94280,La adherencia al miedo,mexico,2020-12-10 21:00:00,https://www.ejecentral.com.mx/la-adherencia-al...,1,0.480863
93811,94281,¿Qué pasó con los sobrevivientes de Ayotzinapa...,mexico,2024-09-24 21:00:00,https://www.bbc.com/mundo/articles/c1e8g5vw864o,1,0.258923


In [7]:
import pandas as pd
import re

print("Limpiando y arreglando el DataFrame...")
print(f"Filas antes de limpiar: {len(df_results)}")

# Función para limpiar texto y remover caracteres problemáticos
def clean_text(text):
    if pd.isna(text):
        return text
    
    # Convertir a string si no lo es
    text = str(text)
    
    # Remover caracteres de control y no imprimibles
    text = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', text)
    
    # Remover comillas dobles problemáticas
    text = text.replace('"', '""')  # Escapar comillas dobles para CSV
    
    # Remover saltos de línea y tabs
    text = text.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    
    # Remover espacios múltiples
    text = re.sub(r'\s+', ' ', text)
    
    # Remover espacios al inicio y final
    text = text.strip()
    
    return text

# Crear una copia limpia del DataFrame
df_clean = df_results.copy()

# Limpiar todas las columnas de texto
text_columns = ['title', 'country', 'url']
for col in text_columns:
    if col in df_clean.columns:
        print(f"Limpiando columna: {col}")
        df_clean[col] = df_clean[col].apply(clean_text)

# Verificar y limpiar la columna 'id' si es necesaria
if 'id' in df_clean.columns:
    print("Limpiando columna: id")
    df_clean['id'] = df_clean['id'].apply(lambda x: str(x).strip() if pd.notna(x) else x)

# Verificar que las columnas numéricas están correctas
print("Verificando columnas numéricas...")
print(f"Valores únicos en 'stars': {sorted(df_clean['stars'].dropna().unique())}")
print(f"Rango de 'score': {df_clean['score'].min():.3f} - {df_clean['score'].max():.3f}")

# Verificar que las fechas están en formato correcto
print("Verificando fechas...")
print(f"Primeras 5 fechas: {df_clean['date'].head().tolist()}")

# Verificar la estructura final
print(f"\nEstructura del DataFrame limpio:")
print(f"Columnas: {list(df_clean.columns)}")
print(f"Filas: {len(df_clean)}")
print(f"Valores nulos por columna:")
print(df_clean.isnull().sum())

# Guardar el DataFrame limpio
output_filename = 'sentiment_analysis_results_clean.csv'
df_clean.to_csv(output_filename, index=False, encoding='utf-8', quoting=1)  # quoting=1 para escapar campos con comas

print(f"\nArchivo limpio guardado como: {output_filename}")

# Mostrar una muestra de los datos limpios
print(f"\nMuestra de datos limpios:")
print(df_clean[['id', 'title', 'stars', 'score']].head())

# Verificar que el archivo se puede leer correctamente
print(f"\nVerificando que el archivo se puede leer correctamente...")
try:
    df_test = pd.read_csv(output_filename)
    print(f"✓ Archivo leído correctamente")
    print(f"✓ Columnas: {list(df_test.columns)}")
    print(f"✓ Filas: {len(df_test)}")
    
    # Verificar que las columnas están en el orden correcto
    expected_columns = ['id', 'title', 'country', 'date', 'url', 'stars', 'score']
    if list(df_test.columns) == expected_columns:
        print("✓ Columnas en el orden correcto")
    else:
        print(f"⚠ Orden de columnas: {list(df_test.columns)}")
        
except Exception as e:
    print(f"✗ Error al leer el archivo: {e}")

Limpiando y arreglando el DataFrame...
Filas antes de limpiar: 93813
Limpiando columna: title
Limpiando columna: country
Limpiando columna: url
Limpiando columna: id
Verificando columnas numéricas...
Valores únicos en 'stars': [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]
Rango de 'score': 0.210 - 0.977
Verificando fechas...
Primeras 5 fechas: ['2021-04-28 20:00:00', '2020-07-02 20:00:00', '2021-05-02 20:00:00', '2021-06-28 20:00:00', '2023-09-13 21:00:00']

Estructura del DataFrame limpio:
Columnas: ['id', 'title', 'country', 'date', 'url', 'stars', 'score']
Filas: 93813
Valores nulos por columna:
id         0
title      0
country    0
date       0
url        0
stars      0
score      0
dtype: int64

Archivo limpio guardado como: sentiment_analysis_results_clean.csv

Muestra de datos limpios:
  id                                              title  stars     score
0  1  Pandemia y salud mental: Ministerio de Ciencia...      4  0.424832
1  2  Podcast, cápsulas anima